In [1]:
from utils.sample import Sample
from scipy.optimize import brentq
import pandas as pd
import numpy as np
from utils.biseccion import bisec
from utils.black_scholes import raiz_ratio

In [2]:
opn = Sample(ratio=[0.4, 1.6], T=[0.2, 1.1], r=[0.02, 0.1], o=[0.01, 1])

opn.create('prueba', N=10**5)
x_test, y_test = opn.open('prueba')

prueba is done ...


In [3]:
df = pd.DataFrame(x_test, columns=['c/k', 'ratio', 'r', 'T'])
df['o'] = y_test
df.head()

,c/k,ratio,r,T,o
0,0.441499,1.371435,0.062964,0.895456,0.305334
1,0.257882,0.767202,0.087680,1.063099,0.983711
2,0.003067,0.480338,0.051090,1.019323,0.364317
3,0.678499,1.432576,0.025423,0.924455,0.930406
4,0.481538,1.462461,0.069658,0.204219,0.511008


In [4]:
# Tiro los casos en que no se puede aplicar el metodo de biseccion
# y calculo la volatilidad implícita
vol = []
i = 0
drops = []
for c, ratio, r, T in x_test:
    f = lambda x: raiz_ratio(c, ratio, r, x, T)
    # que se cumpla la precondicción
    if f(0.01) < 0:
        # máxima precision
        o = bisec(f, 0.01, 1, 2**-56)
        vol.append(o)
    else: 
        drops.append(i)
    i += 1
    
    if i % 10000 == 0:
        print('{}%'.format(100*i/len(x_test)))
    
    
dfb = df.drop(drops)
#volatilidad implicita
dfb['o_est'] = vol

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [5]:
# error entre la volatilidad estimada e implícita
dfb['diff'] = (dfb['o'] - dfb['o_est']).apply(abs)
print('Error absoluto medio biseccion: ', dfb['diff'].mean())

Error absoluto medio biseccion:  2.758172662768739e-06


In [6]:
#Ordeno el dataframe segun el error entre la volatilidad estimada
# y la volatilidad implícita
orda = dfb.sort_values('diff', ignore_index=True, ascending= False)

TypeError: sort_values() got an unexpected keyword argument 'ignore_index'

In [ ]:
#aplico la función que busca la raiz sobre la volatilidad generada
#mediante el método de bisección
fs = []
auxi = orda[['c/k','ratio','r', 'o_est', 'T']]
for i in range(len(auxi)):
    fs.append(raiz_ratio(*auxi.iloc[i]))

orda['f(o_est)'] = fs


In [ ]:
orda.head(50)

Observar q en muchos casos la diferencia entre la volatilidad implícita y la volatilidad estimada es grande, eso es porque en ciertos casos la volatilidad tiene poco impacto, osea un vega bajo, y por el problema de precisión la función que calcula la raíz retorna 0.